# 이미지 준비

In [ ]:
%%bash
apt install -y imagemagick
[ ! -f flower_photos_300x200_small_train_test.zip ]&& wget https://raw.githubusercontent.com/Finfra/AI_CNN_RNN/main/data/flower_photos_300x200_small_train_test.zip

rm -rf __MACOSX
rm -rf flowers
unzip -q flower_photos_300x200_small_train_test.zip
mv flower_photos_300x200_small_train_test flowers

cd flowers
files=$(find |grep "\.jpg$\|\.png$")
for i in $files; do
    convert $i -quiet  -resize 300x200^ -gravity center -extent 300x200  -colorspace Gray    ${i%.*}.png
    #convert $i -quiet  -resize 300x200^ -gravity center -extent 300x200  -define png:color-type=2   ${i%.*}.png
    
    # identify ${i%.*}.png
    rm -f $i
done

find .|grep .DS_Store|xargs rm -f
find .|head -n 10


# CNN Example FromFile by Keras


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

import numpy as np


# 해당 Path의 이미지 Array로 리턴

In [ ]:
from os import listdir
from os.path import isfile, join
from pylab import *
from numpy import *

def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]


def getImagesAndLabels(tPath,isGray=False):
    labels=getFolder(tPath,False)
    tImgDic={f:getFolder(join(tPath,f)) for f in labels}
    tImages,tLabels=None,None
    ks=sorted(list(tImgDic.keys()))

    oh=np.identity(len(ks))
    for label in tImgDic.keys():
        for image in tImgDic[label]:
            le=np.array([float(label)],ndmin=1)
            img=imread(join(tPath,label,image))
            if isGray:
                img=img.reshape(img.shape+(1,))
            
            img=img.reshape((1,) +img.shape)
            
            if tImages is None:
                tImages, tLabels =img, le
            else:
                tImages,tLabels = np.append(tImages,img,axis=0), np.append(tLabels,le ,axis=0)
    return (tImages,to_categorical(tLabels) )

    

In [ ]:
w=300
h=200
color=1


In [ ]:

tPath='flowers/train'
train_images,train_labels=getImagesAndLabels(tPath,isGray=True)    
tPath='flowers/test'
test_images,test_labels=getImagesAndLabels(tPath,isGray=True)    
train_images, test_images = train_images / 255.0, test_images / 255.0

print(f"Shape of Train_images = {train_images.shape} , Shape of Train_labels = {train_labels.shape}")


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(96, (13, 13), activation='relu', padding='same', input_shape=(h,w, color)))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((3, 3)))


model.add(layers.Conv2D(64, (7, 7), activation='relu', padding='same'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Flatten())

model.add(layers.Dense(200, activation='tanh',kernel_regularizer='l1'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()



In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

hist=model.fit(train_images, train_labels, epochs=40,batch_size=100)

# Train history

In [ ]:

print('## training loss and acc ##')
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper right')

plt.show()



# Result

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0, batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])